In [16]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, classification_report
from torch import optim, nn
import wandb
from tqdm import tqdm

wandb.login()

True

In [17]:
config = {
    "learning_rate": 0.01,
    "architecture": "NN",
    "dataset": "static_1.1",
    "epochs": 5,
    "classes": 2,
    "batch_size": 32,
    "num_layers": 2,
    "hidden_size": 128,
    "dropout_prob": 0.3,
    "input_size": 99,
    "output_size": 1,
    "optimizer": "Adam",
    "loss": "CrossEntropyLoss",
    "activation": "ReLU",
}

In [18]:
def model_pipeline(hyperparameters):
    with wandb.init(project="leaguify", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, val_loader, test_loader, criterion, optimizer = make(config)
        print(model)

        # and use them to train the model
        train(model, train_loader, val_loader, criterion, optimizer, config)

        # and test its final performance
        test(model, val_loader)
        test(model, test_loader)

    return model

In [19]:
class StaticDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, device=device)
        self.labels = torch.tensor(np.load(data_dir)[:, -1], dtype=torch.int64, device=device)
        self.transform = transform
        self.target_transform = target_transform
        self.print_statistics()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx, 1:]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label

    def print_statistics(self):
        print(f'Number of samples: {len(self.data)}')
        print(f'Number of features: {len(self.data[0])}')
        print(f'Number of classes: {len(np.unique(self.labels.cpu().numpy()))}')
        print(f'Number of samples per class: {np.bincount(self.labels.cpu().numpy())}')

In [20]:
def make(config):
    train_data, val_data = get_train_data()
    train_loader = make_loader(train_data, batch_size=config.batch_size)
    val_loader = make_loader(val_data, batch_size=config.batch_size)
    test_loader = make_loader(get_test_data(), batch_size=config.batch_size)
    model = NeuralNetwork(config.input_size, config.hidden_size, config.num_layers, config.dropout_prob,
                          config.classes).to(device)
    criterion = nn.CrossEntropyLoss()
    if config.optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=config.learning_rate)
    else:
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    return model, train_loader, val_loader, test_loader, criterion, optimizer

In [21]:
def get_train_data(val_split=0.8):
    dataset = StaticDataset('../data/processed/train_static.npy')
    train_len = int(len(dataset) * val_split)
    val_len = len(dataset) - train_len
    print(f'train_len: {train_len}, val_len: {val_len}')
    return torch.utils.data.random_split(dataset, [train_len, val_len])

In [22]:
def get_test_data():
    return StaticDataset('../data/processed/test_static.npy')

In [23]:
def make_loader(dataset, batch_size=64):
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [24]:
device = (
    "cuda" if torch.cuda.is_available()
    else "cpu"
)
if torch.cuda.is_available():
    print(f'PyTorch version: {torch.__version__}')
    print('*' * 10)
    print(f'_CUDA version: ')
    !nvcc --version
    print('*' * 10)
    print(f'CUDNN version: {torch.backends.cudnn.version()}')
    print(f'Available GPU devices: {torch.cuda.device_count()}')
    print(f'Device Name: {torch.cuda.get_device_name()}')
print(f"Using {device} device")

PyTorch version: 2.1.0+cu121
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:09:35_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
**********
CUDNN version: 8801
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 2080
Using cuda device


In [25]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, num_classes=2):
        super(NeuralNetwork, self).__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential()
        for i in range(num_layers):
            if i == 0:
                self.linear_relu_stack.append(nn.Linear(input_size, hidden_size))
            else:
                self.linear_relu_stack.append(nn.Linear(hidden_size, hidden_size))
            self.linear_relu_stack.append(nn.ReLU())
            self.linear_relu_stack.append(self.dropout)
        self.linear_relu_stack.append(nn.Linear(hidden_size, num_classes))
        self.linear_relu_stack.append(nn.Sigmoid())

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [26]:
def train(model, train_loader, val_loader, criterion, optimizer, config):
    wandb.watch(model, criterion, log="all", log_freq=10)
    example_count = 0
    batch_count = 0
    for epoch in tqdm(range(config.epochs)):
        model.train()
        for _, (matches, labels) in enumerate(train_loader):
            matches, labels = matches.to(device), labels.to(device)
            loss = train_batch(matches, labels, model, optimizer, criterion)
            example_count += len(matches)
            batch_count += 1
            if ((batch_count + 1) % 25) == 0:
                train_log(loss, example_count, epoch)

In [27]:
def train_batch(matches, labels, model, optimizer, criterion):
    matches, labels = matches.to(device), labels.to(device)

    # Forward pass ➡
    outputs = model(matches)
    loss = criterion(outputs, labels)

    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [28]:
def train_log(loss, example_count, epoch):
    wandb.log({"epoch": epoch, "loss": loss}, step=example_count)
    print(f"Loss after {str(example_count).zfill(5)} examples: {loss:.3f}")

In [29]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        y_pred = []
        y_true = []
        for matches, labels in test_loader:
            matches, labels = matches.to(device), labels.to(device)
            outputs = model(matches)
            _, predicted = torch.max(outputs.data, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(classification_report(y_pred, y_true))

        print(f"Accuracy of the model on the {total} " +
              f"test matches: {correct / total:%}")

        wandb.log({"test_accuracy": correct / total})

In [30]:
model = model_pipeline(config)

Number of samples: 15960
Number of features: 100
Number of classes: 2
Number of samples per class: [7591 8369]
train_len: 12768, val_len: 3192
Number of samples: 3990
Number of features: 100
Number of classes: 2
Number of samples per class: [1902 2088]
NeuralNetwork(
  (dropout): Dropout(p=0.3, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=99, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=128, out_features=2, bias=True)
    (7): Sigmoid()
  )
)


  0%|          | 0/5 [00:00<?, ?it/s]

Loss after 00768 examples: 0.641
Loss after 01568 examples: 0.672
Loss after 02368 examples: 0.585
Loss after 03168 examples: 0.568
Loss after 03968 examples: 0.574
Loss after 04768 examples: 0.561
Loss after 05568 examples: 0.472
Loss after 06368 examples: 0.566
Loss after 07168 examples: 0.673
Loss after 07968 examples: 0.603
Loss after 08768 examples: 0.724
Loss after 09568 examples: 0.585
Loss after 10368 examples: 0.547
Loss after 11168 examples: 0.619
Loss after 11968 examples: 0.627


 20%|██        | 1/5 [00:01<00:05,  1.28s/it]

Loss after 12768 examples: 0.698
Loss after 13568 examples: 0.665
Loss after 14368 examples: 0.543
Loss after 15168 examples: 0.569
Loss after 15968 examples: 0.675
Loss after 16768 examples: 0.596
Loss after 17568 examples: 0.677
Loss after 18368 examples: 0.536
Loss after 19168 examples: 0.572
Loss after 19968 examples: 0.563
Loss after 20768 examples: 0.695
Loss after 21568 examples: 0.604
Loss after 22368 examples: 0.632
Loss after 23168 examples: 0.606
Loss after 23968 examples: 0.649


...

 80%|████████  | 4/5 [00:04<00:01,  1.22s/it]

Loss after 50368 examples: 0.581
Loss after 51168 examples: 0.710
Loss after 51968 examples: 0.737
Loss after 52768 examples: 0.526
Loss after 53568 examples: 0.725
Loss after 54368 examples: 0.549
Loss after 55168 examples: 0.581
Loss after 55968 examples: 0.588
Loss after 56768 examples: 0.710
Loss after 57568 examples: 0.624
Loss after 58368 examples: 0.587
Loss after 59168 examples: 0.709
Loss after 59968 examples: 0.673
Loss after 60768 examples: 0.634
Loss after 61568 examples: 0.599
Loss after 62368 examples: 0.829
Loss after 63168 examples: 0.568


100%|██████████| 5/5 [00:06<00:00,  1.24s/it]


              precision    recall  f1-score   support

           0       0.62      0.64      0.63      1480
           1       0.68      0.66      0.67      1712

    accuracy                           0.65      3192
   macro avg       0.65      0.65      0.65      3192
weighted avg       0.65      0.65      0.65      3192

Accuracy of the model on the 3192 test matches: 65.256892%
              precision    recall  f1-score   support

           0       0.61      0.64      0.63      1821
           1       0.69      0.66      0.67      2169

    accuracy                           0.65      3990
   macro avg       0.65      0.65      0.65      3990
weighted avg       0.65      0.65      0.65      3990

Accuracy of the model on the 3990 test matches: 65.037594%


epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
loss,▅▄▃▁▆▇▃▅▅▃▄▂▃▄▄▅▅▆▅▄▄█▇▇▅▆█▆▄▇▆▃▇▇▃▆▄▆▄▃
test_accuracy,█▁
epoch,4
loss,0.56763
test_accuracy,0.65038
